In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import zipfile
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
from PIL import Image

In [ ]:
from google.colab import files
uploaded = files.upload()

In [2]:
zip_file_name = 'diatom_dataset.zip'

extract_path = '/content/dataset'

zip_file_path = f'/content/{zip_file_name}'

if not os.path.exists(extract_path):
    os.makedirs(extract_path)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
  zip_ref.extractall(extract_path)
print("압축 해제 완료")

압축 해제 완료


In [3]:
!rm -rf /content/dataset/__MACOSX

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [7]:
base_dir = "/content/dataset/diatom_dataset"

species_folders = [os.path.join(base_dir, f) for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
genus_groups = {}

for path in species_folders:
    folder_name = os.path.basename(path)
    genus = folder_name.split(' ')[0]  # 예: "Achnanthes coarctata" → "Achnanthes"
    genus_groups.setdefault(genus, []).append(path)

print(f"총 속 개수: {len(genus_groups)}")
print(genus_groups.keys())

총 속 개수: 12
dict_keys(['Amphora', 'Halamphora', 'Fragilariforma', 'Geissleria', 'Diatoma', 'Lindavia', 'Fragilaria', 'Epithemia', 'Anomoeoneis', 'Achnanthes', 'Chamaepinnularia', 'Caloneis'])


In [8]:
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

In [10]:
def train_expert(genus_name):

    species_dirs = [d for d in os.listdir(base_dir) if d.startswith(genus_name)]

    image_paths, labels = [], []
    for label_idx, folder in enumerate(species_dirs):
        folder_path = os.path.join(base_dir, folder)
        if not os.path.isdir(folder_path):
            continue
        for img_name in os.listdir(folder_path):
            if img_name.lower().endswith((".jpg", ".jpeg", ".png")):
                image_paths.append(os.path.join(folder_path, img_name))
                labels.append(label_idx)

    print(f"\n[{genus_name}] 이미지 {len(image_paths)}장, 종 {len(species_dirs)}개")

    # train/test
    full_trainset = SimpleDataset(image_paths, labels, transform=train_transform)
    full_testset  = SimpleDataset(image_paths, labels, transform=test_transform)

    train_size = int(0.8 * len(image_paths))
    test_size  = len(image_paths) - train_size

    train_dataset, _ = random_split(full_trainset, [train_size, test_size],
                                generator=torch.Generator().manual_seed(42))
    _, test_dataset  = random_split(full_testset, [train_size, test_size],
                                generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False)

    # model
    model = models.resnet18(pretrained=True)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, len(species_dirs))
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # 학습
    EPOCHS = 15
    for epoch in range(EPOCHS):
        model.train()
        total, correct = 0, 0
        for images, labels in tqdm(train_loader, desc=f"[{genus_name}] Epoch {epoch+1}/{EPOCHS}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        print(f"{genus_name} Train Acc: {100 * correct / total:.2f}%")

    # 평가
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = 100 * correct / total
    print(f"{genus_name} Expert Test Acc: {acc:.2f}%")

    os.makedirs("experts", exist_ok=True)
    torch.save(model.state_dict(), f"experts/{genus_name}_expert.pth")

    return {"Genus": genus_name, "NumSpecies": len(species_dirs), "Accuracy": acc}

genus_list = sorted(set([
    name.split(" ")[0] for name in os.listdir(base_dir)
    if not name.startswith(".") and os.path.isdir(os.path.join(base_dir, name))
]))
print("발견된 속 목록:", genus_list)

results = []
for genus in genus_list:
    result = train_expert(genus)
    if result:
        results.append(result)

df = pd.DataFrame(results)
df.to_csv("expert_results.csv", index=False)
print("\nExpert 모델 학습 완료!")
display(df)

발견된 속 목록: ['Achnanthes', 'Amphora', 'Anomoeoneis', 'Caloneis', 'Chamaepinnularia', 'Diatoma', 'Epithemia', 'Fragilaria', 'Fragilariforma', 'Geissleria', 'Halamphora', 'Lindavia']

[Achnanthes] 이미지 50장, 종 7개
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:00<00:00, 229MB/s]
[Achnanthes] Epoch 1/15: 100%|██████████| 3/3 [00:01<00:00,  1.56it/s]


Achnanthes Train Acc: 12.50%


[Achnanthes] Epoch 2/15: 100%|██████████| 3/3 [00:00<00:00,  6.50it/s]


Achnanthes Train Acc: 55.00%


...

[Lindavia] Epoch 14/15: 100%|██████████| 5/5 [00:00<00:00,  5.28it/s]


Lindavia Train Acc: 100.00%


[Lindavia] Epoch 15/15: 100%|██████████| 5/5 [00:00<00:00,  5.20it/s]


Lindavia Train Acc: 100.00%
Lindavia Expert Test Acc: 88.89%

Expert 모델 학습 완료!


,Genus,NumSpecies,Accuracy
0,Achnanthes,7,90.000000
1,Amphora,7,81.818182
2,Anomoeoneis,6,80.000000
3,Caloneis,5,100.000000
4,Chamaepinnularia,7,90.909091
5,Diatoma,5,62.500000
6,Epithemia,10,71.428571
7,Fragilaria,10,64.705882
8,Fragilariforma,8,84.615385
9,Geissleria,8,38.461538
